In [1]:
import serial
import os
import configparser


class RPI4_to_SEW:
    def __init__(self, nodename, port, timeout=None, write_timeout=None, config_path="/config/config.ini"):
        self._nodename = nodename
        self._config_path = config_path
        self._inverters = []
        self.config = configparser.ConfigParser()
        # self.serial = serial.Serial(port=port, baudrate=9600, parity=serial.PARITY_EVEN,
        #                            stopbits=serial.STOPBITS_ONE, timeout=timeout, write_timeout=write_timeout, exclusive=True)

    def populate_vfds(self):
        #full_path = os.path.realpath(__file__)
        #dir_path = os.path.dirname(full_path)
        #config_path = dir_path + self._config_path

        self.config.read('')

        for vfd_config in self.config:
            sew_vfd = self.SEW_VFD(vfd_config)
            self._inverters.append(sew_vfd)

    def list_vfds(self):
        vfd_list = []
        for vfd in self._inverters:
            vfd_list.append(vfd.name)
        return vfd_list

    def get_config(self):
        return self.config

    class SEW_VFD:
        def __init__(self, config):
            print(f'creating vfd with config: {config}')
            self.config = config
            self.name = 'Test'

        def ramp_to_coded(self, ramp=2):

            ramp = ramp * 1000
            coded_ramp = ramp.to_bytes(2, "big", signed=True)

            return coded_ramp

        def speed_to_coded(self, speed):

            speed_temp = round(speed / 0.0061)
            print(f"dec speed: {speed_temp}")
            coded_speed = speed_temp.to_bytes(2, "big", signed=True)

            return coded_speed

        def append_bcc(self, test):
            checksum = 0
            for el in test:
                checksum ^= el

            test.append(checksum)
            return checksum


rpi = RPI4_to_SEW('test', 0, 1, 1)
rpi.populate_vfds()
print(rpi.list_vfds())

config = rpi.get_config()

# start = bytearray(
#     [
#         0x02,
#         0x00,
#         0x85,
#         0x00,
#         0x06,
#         coded_speed[0],
#         coded_speed[1],
#         coded_ramp[0],
#         coded_ramp[1],
#     ]
# )
# stop = bytearray([0x02, 0xFF, 0x85, 0x00, 0x00, 0x20, 0x00, 0x0B, 0xB8])

# append_bcc(start)
# print(start)

# print(start)


creating vfd with config: DEFAULT
['Test']


In [2]:
def tohex(val, nbits):
  return hex((val + (1 << nbits)) % (1 << nbits))


val = +100
val2 = +100

test1 = val.to_bytes(2, "big")
test2 = val2.to_bytes(2, 'big', signed=True)

for byte in test2:
  print(hex(byte))


0x0
0x64


In [3]:
def speed_to_coded(speed):

    py_clip = lambda x, l, u: l if x < l else u if x > u else x

    speed_temp = round(speed / 0.0061)
    speed_temp = py_clip(speed_temp, -16384, 16384)
    print(f"dec speed: {speed_temp}")
    coded_speed = speed_temp.to_bytes(2, "big", signed=True)

    for byte in coded_speed:
        print(hex(byte))

speed_to_coded(-100)

dec speed: -16384
0xc0
0x0


In [4]:
def ramp_to_coded(ramp=None):

    if ramp == None:
        raise Exception('Ramp value is empty')

    temp_ramp = int(ramp * 1000)

    coded_ramp = temp_ramp.to_bytes(2, "big", signed=True)

    for byte in coded_ramp:
        print(hex(byte))

ramp_to_coded(0.1)

0x0
0x64


In [5]:
stop = bytearray([0x02, 0xFF, 0x85, 0x00, 0x00, 0x20, 0x00, 0x0B, 0xB8])  

def print_packet_as_hex(packet):

    strpacket = '-'.join(hex(x) for x in packet)
   
    return strpacket

print(display_packet(stop))
    # for hex_val in packets[0]:
    #     print(hex(hex_val))

NameError: name 'display_packet' is not defined

In [ ]:
out=bytearray([0x2,0x1,0x5,0x0,0x0,0x40,0x0,0x0,0x64,0x22])
incoming=bytearray([0x1d,0x1,0x5,0x4,0x6,0x3,0x0,0x6,0x7,0x19])

In [3]:
import utils

incoming=bytearray([0x1d,0x1,0x5,0x4,0x6,0x3,0x0,0x6,0x7,0x19])

utils.parse_status_packet(incoming)

sender: vfd, address: 1, comm_type: 5, sw1: 1030, curr: 768, sw2: 1543, bcc: valid


In [ ]:
import utils

cw1 = 6

test = utils.ControlCommands(cw1)